In [55]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import re
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

def search_books(key='한글'):
# 환경 변수에서 값 읽기
    client_id = os.getenv("NAVER_CLIENT_ID")
    client_secret = os.getenv("NAVER_CLIENT_SECRET")

    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret,
    }

    payload = {
        'query': key,
        'display': 50,
        'sort': 'date'
    }

    url = 'https://openapi.naver.com/v1/search/book.json'

    # 요청 보내기
    res = requests.get(url, params=payload, headers=headers)
    items_data = res.json()['items']

    # 결과 저장 리스트
    items_list = list()
    item_list = []


    for i, item in enumerate(items_data):
        title = re.sub(r'<.*?>', '', item['title'])
        description = re.sub(r'<.*?\ㅜ>', '', item['description']).replace('\n', ' ')

        item_list.append(title)
        item_list.append(item['link'])
        item_list.append(item['image'])
        item_list.append(item['author'])
        item_list.append(item['pubdate'])
        item_list.append(item['discount'])
        item_list.append(item['publisher'])
        item_list.append(item['isbn'])
        item_list.append(description)

        items_list.append(item_list)
        item_list = [] 

    df = pd.DataFrame(
        items_list,
        columns=["title", "link", "image", "author", "pubdate", "discount", "publisher", "isbn", "description"]
    )

    # 저장 ->ai의 힘을 빌렸습니다..
    df.to_json('data/books.json', orient='records', force_ascii=False, indent=2)

    # 불러오기
    book_df = pd.read_json('data/books.json')
        # 출력
    print("===도서목록===")
    print(type(book_df.head()))
    print(book_df.head())
    print(f"받은 결과 수: {len(items_data)}")



search_books('파이썬')


===도서목록===
<class 'pandas.core.frame.DataFrame'>
                                               title  \
0  2025 시대에듀 빅데이터분석기사 실기(파이썬) 한권으로 끝내기 (한국데이터산업진흥...   
1                   생물정보학 실무 가이드 : 바이오파이썬과 데이터베이스 활용   
2  선생님을 위한 8282 업무 자동화 with AI + 파이썬 + 노코드 (학교 수업...   
3  코딩 자율학습  잔재미코딩의 파이썬 데이터 분석 입문 (Pandas, Plotly ...   
4  제대로 시작하는 챗GPT와 AI 활용 with 파이썬 (프롬프트 엔지니어링, 음성 ...   

                                                link  \
0  https://search.shopping.naver.com/book/catalog...   
1  https://search.shopping.naver.com/book/catalog...   
2  https://search.shopping.naver.com/book/catalog...   
3  https://search.shopping.naver.com/book/catalog...   
4  https://search.shopping.naver.com/book/catalog...   

                                               image              author  \
0  https://shopping-phinf.pstatic.net/main_532452...                 장희선   
1  https://shopping-phinf.pstatic.net/main_532637...                 도진환   
2  https://shopping-phinf.pstatic.net/main_534907